# Fleming Pipeline - ipython notebook version

### Note on astrometry.net 

Shouldn't need a key to find RA/DEC coordinates since `force_upload=False`.

If the solve keeps timing out and you have `Config.astrometry_timeout` as a large value (or `-1`) then check the timeout in `~/.astroquery/config/astroquery.cfg`.


#### How to get a key
If you do want to upload (eg if machine takes too long solving) you will need a key and add it to `~/.astroquery/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`.
The first is a config file (example layout is as below) latter is just a text file containing the key and nothing else.

[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Example config:
```
[astrometry_net]

api_key = XXXXX
timeout = 1200 ## 20 mins
server = http://npva.astrometry.net
```

## General imports and setup
Run this, change the config cell to suit the field that you want to run for.

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports
import pipeline
from pipeline import *
from pipeline import Pipeline

from datetime import datetime
import os

In [ ]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

## A4 paper
mpl.rcParams['figure.figsize'] = [11.3, 8.7]

In [ ]:
config = Config(
    raw_image_dir = os.path.expanduser("~/mnt/jgt/2022/0301"),
    image_prefix = "l138_0",
    bias_prefix = "bias",
    n_sets = 9,
    fits_extension = ".fits",
    fits_date_format = "%Y.%m.%dT%H:%M:%S.%f",
    has_filter_in_header = False,
)

## Run the pipeline
Run using the general setup from `pipeline/Pipeline.py`.

In [ ]:
Pipeline.run(config, show_plots=True, show_errors=False)

## Pipeline breakdown
Below is a rough copy of the `Pipeline.run()` method.

Used for picking certain parts to run/rerun.
Hopefully minimal kernel restarts are needed when running.

In [ ]:
r = Reducer(config, "No filter") ## Only "No filter" for Trius

r.reduce(skip_existing=True)

In [ ]:
c = Cataloguer(config)

catalogue_image = os.path.join(config.image_dir, config.image_format_str.format(1, 1))

In [ ]:
#n_sources = c.generate_catalogue(catalogue_image, solve=True)
#c.generate_image_times()

n_sources = c.generate_catalogue(catalogue_image, solve=False)

In [ ]:
sf = ShiftFinder(config, n_sources)

sf.generate_shifts()

In [ ]:
ff = FluxFinder(config, n_sources)

In [ ]:
ff.make_light_curves()

In [ ]:
da = DataAnalyser(config)

In [ ]:
da.create_avg_curve()
ff.plot_avg_light_curve(config.avg_curve_path, show=True)

In [ ]:
ff.create_adjusted_light_curves()

In [ ]:
variable_ids = da.get_variables()
ff.plot_given_light_curves(variable_ids, adjusted=True, show=True, show_errors=False)

In [ ]:
da.output_results()
da.create_thumbnails(ff)

## Debug land

Use at your own peril.

If you change something in the `pipeline/*.py` then you need to reload the modules differently instead of with the normal imports.

In [1]:
%load_ext autoreload
%autoreload 2

## Imports
import pipeline
from pipeline import *
from pipeline import Pipeline

from datetime import datetime
import os

In [2]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

## A4 paper
mpl.rcParams['figure.figsize'] = [11.3, 8.7]

In [3]:
config = Config(
    raw_image_dir = os.path.expanduser("~/mnt/jgt/2022/0301"),
    image_prefix = "l138_0",
    bias_prefix = "bias",
    n_sets = 9,
    fits_extension = ".fits",
    fits_date_format = "%Y.%m.%dT%H:%M:%S.%f",
    has_filter_in_header = False,
)

In [9]:
## DataAnalyser, un-adjusted
da = DataAnalyser(config, adjusted=False)

mean, std, med, n_positive = da.get_means_and_stds()
source_ids = da.get_source_ids()
da.plot_means_and_stds()
    
vd = VariableDetector(config, source_ids, mean, std, med, n_positive, adjusted=False)
exclude_ids = vd.std_dev_search(config.avg_exclude_threshold, adjusted=False)
avg_ids = da.get_ids_for_avg(exclude_ids)

da.make_avg_curve(avg_ids)
ff.plot_avg_light_curve(config.avg_curve_path, show=True)



[DEBUG] listing sources
[DEBUG] listing sources
[VariableDetector] std search: Found 111 variables out of 1743 sources
[DEBUG] listing sources
[DEBUG] Calling `make_avg_curve` in DataAnalyser
[DEBUG] listing sources


NameError: name 'ff' is not defined

In [ ]:
Pipeline.run_existing(config, assume_already_adjusted=True, show_plots=True, show_errors=False)